# Setup

In [0]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%%bash
[[ ! -e /tools/google-cloud-sdk ]] && exit
echo "Installing Dependencies ..."
sudo apt-get update
sudo apt-get install -y -q llvm-6.0 libglfw3-dev libtinfo-dev libffi-dev zlib1g-dev clinfo
cd "/content/gdrive/My Drive"
if [[ ! -e tvm ]]; then
    echo "Cloning TVM ..."
    git clone --recursive https://github.com/dmlc/tvm
fi
echo "Configuring Build ..."
cd tvm
mkdir -p build
cp cmake/config.cmake build
# sed -i -e 's/USE_OPONGL OFF/USE_OPONGL ON/g' build/config.cmake
sed -i -e 's/USE_CUDA OFF/USE_CUDA ON/g' build/config.cmake
sed -i -e 's/USE_CUDNN OFF/USE_CUDNN ON/g' build/config.cmake
sed -i -e 's/USE_LLVM OFF/USE_LLVM ON/g' build/config.cmake
sed -i -e 's/USE_VTA_TSIM OFF/USE_VTA_TSIM ON/g' build/config.cmake
cat build/config.cmake
echo "Running CMake ..."
cd build
cmake ..
echo "Building TVM ..."
make -j4
cd ..
echo "Installing Python libraries ..."
cd "/content/gdrive/My Drive/tvm/"
cd python; python setup.py install; cd ..
cd topi/python; python setup.py install; cd ..; cd ..
export TVM_HOME=$(eval pwd)/tvm
export PYTHONPATH=$TVM_HOME/python:$TVM_HOME/topi/python:$TVM_HOME/nnvm/python:${PYTHONPATH}

Installing Dependencies ...
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information

bash: line 5: cd: /content/gdrive/My Drive: No such file or directory
CMake Warning (dev) at /usr/local/lib/python2.7/dist-packages/cmake/data/share/cmake-3.12/Modules/FindOpenGL.cmake:270 (message):
  Policy CMP0072 is not set: FindOpenGL prefers GLVND by default when
  available.  Run "cmake --help-policy CMP0072" for policy details.  Use the
  cmake_policy command to set the policy and suppress this warning.

  FindOpenGL found both a legacy GL library:

    OPENGL_gl_LIBRARY: /usr/lib/x86_64-linux-gnu/libGL.so

  and GLVND libraries for OpenGL and GLX:

    OPENGL_opengl_LIBRARY: /usr/lib/x86_64-linux-gnu/libOpenGL.so
    OPENGL_glx_LIBRARY: /usr/lib/x86_64-linux-gnu/libGLX.so

  OpenGL_GL_PREFERENCE has not been set to "GLVND" or "LEGACY", so for
  compatibility with CMake 3.10 and below the legacy GL library will be used.
Call Stack (most recent call first):
  cmake/modules/OpenGL.cmake:18 (find_package)
  CMakeLists.txt:192 (include)
This warning is for project developers.  Use 

# MicroTVM


#### TODO: should we add some basic info about MicroTVM itself? People might miss things during the talk.

First, we run some necessary imports.


In [0]:
import os

import numpy as np
import tvm
from tvm.contrib import graph_runtime, util
from tvm import relay
import tvm.micro as micro

ModuleNotFoundError: ignored

## C Codegen

Let's take a look at the C codegen backend for a simple function.

In [0]:
# First, we construct the function.
ty = relay.TensorType(shape=(1024,), dtype="float32")
x = relay.var("x", ty)
y = relay.var("y", ty)
func = relay.Function([x], relay.add(x, y))

# Then build it with the appropriate configuration.
with tvm.build_config(disable_vectorize=True):
  with relay.build_config(opt_level=3):
    _, src_mod, _ = relay.build(func, target="c", params={})

# Now, we can see the source that was generated.
print(src_mod.get_source())

NameError: ignored

Try building some other functions and see what source is generated.

In [0]:
def build_c_module(func):
  with tvm.build_config(disable_vectorize=True):
    with relay.build_config(opt_level=3):
      _, src_mod, _ = relay.build(func, target="c", params={})
  return src_mod


In [0]:
# TODO: Try another function.
func = relay.Function([], relay.const(1.0))

print(build_c_module(func).get_source())

## Graph Runtime Execution

The primary execution for running models with MicroTVM is the graph runtime.  Here's a helper function for building a graph runtime module from a Relay function.

In [0]:
def micro_build_graph_runtime(func, device_type, params={}):
    """Create a graph runtime module with a micro device context."""
    with tvm.build_config(disable_vectorize=True):
        with relay.build_config(opt_level=3):
            graph, host_mod, params = relay.build(func, target="c",
                                                  params=params)

    micro_mod = micro.from_source_module(host_mod, device_type)
    ctx = tvm.micro_dev(0)
    graph_mod = graph_runtime.create(graph, micro_mod, ctx)
    return graph_mod, params

Being able to construct a graph runtime module means we can now plug in existing models and they just work.

In [0]:
from tvm.relay.testing import resnet

resnet_func, params = resnet.get_workload(num_classes=10,
                                          num_layers=18,
                                          image_shape=(3, 32, 32))
# Remove the final softmax layer, because uTVM does not currently support
# it.
resnet_func_no_sm = relay.Function(resnet_func.params,
                                   resnet_func.body.args[0],
                                   resnet_func.ret_type)

# For now, we use the "host" emulated device for execution.  In the next
# section, we'll implement our own simple low-level device.
device_type = "host"
micro.init(device_type)
mod, params = micro_build_graph_runtime(resnet_func_no_sm, device_type,
                                        params=params)
# Set the model weights.
mod.set_input(**params)
# Generate random input.
data = np.random.uniform(size=mod.get_input(0).shape)
mod.run(data=data)
result = mod.get_output(0).asnumpy()
# We gave a random input, so all we want is a result with some nonzero
# entries.
assert result.sum() != 0.0

## The Low-Level Device Interface

The low-level device interface is used by TVM to communicate with MicroDevices. It exposes three important functions to interact with the MicroDevice: 
- Read from device memory.
- Write to device memory.
- Start function execution.


Below is an example which uses an allocated region of memory on the host machine to simulate a MicroDevice. This HostLowLevelDevice provides MicroTVM users with an easy test setup to experiment with, without having to communicate and debug external MicroDevices. However, the LowLevelDevice interface can be implemented for any MicroDevice, for example over JTAG or a network connection, which will enable you to run TVM on your own MicroDevice.


In [0]:
#@title 
#include <sys/mman.h>
#include <cstring>
#include "low_level_device.h"
#include "micro_common.h"

namespace tvm {
namespace runtime {
/*!
 * \brief emulated low-level device on host machine
 */
class HostLowLevelDevice final : public LowLevelDevice {
 public:
  /*!
   * \brief constructor to initialize on-host memory region to act as device
   * \param num_bytes size of the emulated on-device memory region
   */
  explicit HostLowLevelDevice(size_t num_bytes)
    : size_(num_bytes) {
    size_t size_in_pages = (num_bytes + kPageSize - 1) / kPageSize;
    int mmap_prot = PROT_READ | PROT_WRITE | PROT_EXEC;
    int mmap_flags = MAP_ANONYMOUS | MAP_PRIVATE;
    base_addr_ = DevBaseAddr(
      (reinterpret_cast<std::uintptr_t>(
        mmap(nullptr, size_in_pages * kPageSize, mmap_prot, mmap_flags, -1, 0))));
  }

  /*!
   * \brief destructor to deallocate on-host device region
   */
  ~HostLowLevelDevice() {
    munmap(base_addr_.cast_to<void*>(), size_);
  }

  void Write(DevBaseOffset offset,
             void* buf,
             size_t num_bytes) final {
    void* addr = (offset + base_addr_).cast_to<void*>();
    std::memcpy(addr, buf, num_bytes);
  }

  void Read(DevBaseOffset offset,
            void* buf,
            size_t num_bytes) final {
    void* addr = (offset + base_addr_).cast_to<void*>();
    std::memcpy(buf, addr, num_bytes);
  }

  void Execute(DevBaseOffset func_offset, DevBaseOffset breakpoint) final {
    DevAddr func_addr = func_offset + base_addr_;
    reinterpret_cast<void (*)(void)>(func_addr.value())();
  }

  DevBaseAddr base_addr() const final {
    return base_addr_;
  }

  const char* device_type() const final {
    return "host";
  }

 private:
  /*! \brief base address of the micro device memory region */
  DevBaseAddr base_addr_;
  /*! \brief size of memory region */
  size_t size_;
};

const std::shared_ptr<LowLevelDevice> HostLowLevelDeviceCreate(size_t num_bytes) {
  std::shared_ptr<LowLevelDevice> lld =
      std::make_shared<HostLowLevelDevice>(num_bytes);
  return lld;
}
}  // namespace runtime
}  // namespace tvm

# Custom Compilation

#### TODO

This will involve showing how the generated C code is compiled by a device-specific `gcc`. Maybe should immediately follow the C codegen section.